In [42]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix


In [39]:
train_dir = r'C:\Users\shanj\OneDrive\Desktop\images\images\training' 

In [40]:
batch_size = 32
img_height = 150
img_width = 150

In [43]:
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)


In [44]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    shuffle=True,
    batch_size=batch_size,
    class_mode='binary',
    subset="training")

Found 1914 images belonging to 2 classes.


In [45]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    subset="validation",
    class_mode='binary')

Found 478 images belonging to 2 classes.


In [49]:
validation_generator

In [46]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [50]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [51]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/5
59/59 [==============================] - 72s 1s/step - loss: 0.3265 - accuracy: 0.8656 - val_loss: 1.8451 - val_accuracy: 0.5759
Epoch 2/5
59/59 [==============================] - 71s 1s/step - loss: 0.1657 - accuracy: 0.9378 - val_loss: 1.9551 - val_accuracy: 0.4955
Epoch 3/5
59/59 [==============================] - 71s 1s/step - loss: 0.1538 - accuracy: 0.9431 - val_loss: 2.8890 - val_accuracy: 0.5670
Epoch 4/5
59/59 [==============================] - 71s 1s/step - loss: 0.1078 - accuracy: 0.9601 - val_loss: 3.0473 - val_accuracy: 0.5848
Epoch 5/5
59/59 [==============================] - 71s 1s/step - loss: 0.0662 - accuracy: 0.9729 - val_loss: 3.2651 - val_accuracy: 0.4375


In [52]:
validation_dir = r'C:\Users\shanj\OneDrive\Desktop\images\images\testing'

In [53]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [54]:
test_set = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary')

Found 597 images belonging to 2 classes.


In [55]:
y_pred1 = model.predict(test_set, steps=len(test_set))

19/19 [==============================] - 17s 880ms/step


In [57]:
y_pred1=np.where(y_pred1 > 0.5, 1, 0)

In [59]:
y_true = test_set.classes

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred1)

# Print the confusion matrix
print(cm)

[[244  46]
 [ 33 274]]


In [60]:
cl = classification_report(y_true, y_pred1,target_names=test_set.class_indices.keys())
print(cl)

              precision    recall  f1-score   support

        flip       0.88      0.84      0.86       290
     notflip       0.86      0.89      0.87       307

    accuracy                           0.87       597
   macro avg       0.87      0.87      0.87       597
weighted avg       0.87      0.87      0.87       597

